In [1]:
import sys

sys.path.append("../")
from hamilton_neural_network import (
    TrainTestData,
    LatentHamiltonianNeuralNetwork,
)
from hamilton_system import HamiltonianSystem
from pdf_models import NegLogIndepedentGaussians,NegLogTenDimRosenbrock
import tensorflow as tf
import matplotlib.pyplot as plt
from no_u_turn.nuts import NoUTurnSampling

tf.random.set_seed(0)

2.18.0


In [2]:
U = NegLogTenDimRosenbrock()
K = NegLogIndepedentGaussians(
    tf.constant([0.0] * 10), tf.constant([1.0] * 10)
)
q0 = tf.constant([[0.0] * 10])
p0 = tf.random.normal(q0.shape)
T = 120.0
leap_frog_per_unit = 40
num_samples = 40
num_train = int(0.9 * num_samples * leap_frog_per_unit * T)

In [3]:
train_test_data = TrainTestData(
    num_samples, T, leap_frog_per_unit, q0, p0, U=U, K=K
)
samples = train_test_data()
tf.io.write_file("../exps/demo5_train_test_data.txt", tf.io.serialize_tensor(samples))

Generating samples...
Finished generating samples.


In [ ]:
file = tf.io.read_file("../exps/demo5_train_test_data.txt")
train_test_data = tf.io.parse_tensor(file, out_type=tf.float32)
train_test_data = tf.random.shuffle(train_test_data)
train_data = train_test_data[:num_train, :]
test_data = train_test_data[num_train:, :]
print(train_data.shape, test_data.shape)
lhnn = LatentHamiltonianNeuralNetwork(3, 100, 10)
lhnn.build(input_shape=(1, 20))
train_hist, test_hist = lhnn.train(1000, 1000, 5e-5, train_data, test_data, save_dir="../exps/demo5_lhnn.weights.h5", print_every=200)

(172800, 40) (19240, 40)
Training started...
Epoch 0: Train loss 15.3114013671875, Test loss 15.28851318359375.
Epoch 200: Train loss 0.7364346981048584, Test loss 0.7811428308486938.
Epoch 400: Train loss 0.30300626158714294, Test loss 0.3424561619758606.
Epoch 600: Train loss 0.16964831948280334, Test loss 0.20736584067344666.
Epoch 800: Train loss 0.13913118839263916, Test loss 0.18842190504074097.
Epoch 1000: Train loss 0.11209020763635635, Test loss 0.17110970616340637.


In [ ]:
lhnn = LatentHamiltonianNeuralNetwork(3, 100, 3)
lhnn.build(input_shape=(1, 6))
lhnn.load_weights("../exps/demo5_lhnn.weights.h5")
q0 = tf.constant([[0.0] * 10])
nuts = NoUTurnSampling(
    num_samples=125000,
    q0=q0,
    dt=0.025,
    lhnn=lhnn,
    Hamiltonian=HamiltonianSystem(U=U, K=K),
    Delta_lf=100.0,
    Delta_lhnn=10.0,
    num_lf_steps=20,
    j_max=12,
)
nuts(print_every=5000)
q_hist = tf.concat(nuts.q_hist, axis=0)
tf.io.write_file("../exps/demo5_q_hist.txt", tf.io.serialize_tensor(q_hist))